# Figure 3 Circle Heatmap all trans Proteins

This notebook take lists of genes from the enrichment analysis and graphs the correlation in circle heat maps. 

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import math as math
import scipy.stats
import re
import sys 
import statsmodels.stats.multitest

import cptac
import cptac.utils as u
import plot_utils as p

/Users/Lindsey/anaconda3/lib/python3.7/site-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


# Load in File and Subset

Insert list from enrichment analysis and add back proteomics to name

In [2]:
t_cell_genes = ['JUN','NFATC2','PIK3CD','CD3G','PIK3CG','VAV1','MAPK13','ZAP70','CD4','PTPRC','PPP3CC','LCK','GRAP2','PRKCQ','GRB2','LCP2','PTPN6','CARD11']
t_cells_column_names = []
for gene in t_cell_genes:
    gene += "_proteomics"
    t_cells_column_names.append(gene)
len(t_cell_genes)


18


Load appended version of data frame with correlations with FDR correction. Subset down dataframe to only have complement genes

In [3]:
df_FDR_append = pd.read_csv("../Step3.2_combining_pearson_dfs/csv_files/pancan_EGFR_pearson_sig_all_prot_append_FDR.csv")
df_FDR_append = df_FDR_append.drop(['Unnamed: 0'], axis=1)

df_FDR_t_cell = df_FDR_append[df_FDR_append.Comparison.isin(t_cells_column_names)]
df_FDR_t_cell

,Comparison,Correlation,P_value,Cancer Type
2,GRB2_proteomics,-0.610889,1.886384e-11,GBM
83,CD4_proteomics,-0.467479,1.069659e-06,GBM
87,PIK3CD_proteomics,-0.466265,1.150323e-06,GBM
100,LCP2_proteomics,-0.460881,1.582634e-06,GBM
127,VAV1_proteomics,-0.447143,3.486858e-06,GBM
200,PTPRC_proteomics,-0.424958,1.163774e-05,GBM
290,CARD11_proteomics,-0.408163,2.743707e-05,GBM
319,PTPN6_proteomics,-0.402017,3.713191e-05,GBM
363,PIK3CG_proteomics,-0.394048,5.449073e-05,GBM
370,NFATC2_proteomics,-0.392602,5.835717e-05,GBM


In [4]:
df_FDR_t_cell["Correlation"].max()

0.4286790220446001

Make figure size of circles base on p_values and color based on correlation. Non sigificant correlations included. 

In [5]:
p.plotCircleHeatMap(df_FDR_t_cell, "P_value","Correlation","Comparison","Cancer Type")

/Users/Lindsey/anaconda3/lib/python3.7/site-packages/plot_utils/__init__.py:40: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["size2"] = df[circle_var].apply(lambda x: -1*(np.log(x)))
/Users/Lindsey/anaconda3/lib/python3.7/site-packages/plot_utils/__init__.py:41: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['size'] = (df["size2"])*3


Loading BokehJS ...

Load in wide data frame and subset with genes in figure in order to see raw numbers associated with figure 

In [20]:
df_FDR_wide = pd.read_csv("../Step3.2_combining_pearson_dfs/csv_files/pancan_EGFR_all_FDR_wide.csv")
df_FDR_wide = df_FDR_wide.drop(['Unnamed: 0'], axis=1)

df_FDR_wide_t_cell = df_FDR_wide[df_FDR_wide.Comparison.isin(t_cells_column_names)]
df_FDR_wide_t_cell

,Comparison,Correlation_Gbm,P_value_Gbm,Correlation_kidney,P_value_kidney,Correlation_Ovar,P_value_Ovar,Correlation_Brca,P_value_Brca,Correlation_Luad,P_value_Luad,Correlation_hnscc,P_value_hnscc,Correlation_colon,P_value_colon,Correlation_Lscc,P_value_Lscc
2,GRB2_proteomics,-0.610889,1.886384e-11,NaN,NaN,NaN,NaN,NaN,NaN,-0.302439,0.001321,-0.532341,2.559824e-09,NaN,NaN,NaN,NaN
83,CD4_proteomics,-0.467479,1.069659e-06,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-0.417232,6.378812e-06,NaN,NaN,NaN,NaN
87,PIK3CD_proteomics,-0.466265,1.150323e-06,NaN,NaN,NaN,NaN,NaN,NaN,-0.331760,0.000399,NaN,NaN,NaN,NaN,NaN,NaN
100,LCP2_proteomics,-0.460881,1.582634e-06,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-0.390098,2.750943e-05,NaN,NaN,NaN,NaN
127,VAV1_proteomics,-0.447143,3.486858e-06,-0.285693,0.002483,NaN,NaN,NaN,NaN,NaN,NaN,-0.439464,1.745944e-06,NaN,NaN,NaN,NaN
200,PTPRC_proteomics,-0.424958,1.163774e-05,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-0.402397,1.440882e-05,NaN,NaN,NaN,NaN
290,CARD11_proteomics,-0.408163,2.743707e-05,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-0.404820,1.264613e-05,NaN,NaN,NaN,NaN
319,PTPN6_proteomics,-0.402017,3.713191e-05,NaN,NaN,NaN,NaN,NaN,NaN,-0.371990,0.000063,-0.411321,8.866717e-06,NaN,NaN,NaN,NaN
363,PIK3CG_proteomics,-0.394048,5.449073e-05,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-0.402484,1.434139e-05,NaN,NaN,NaN,NaN
370,NFATC2_proteomics,-0.392602,5.835717e-05,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-0.457536,5.679765e-07,NaN,NaN,NaN,NaN


# Killer T cell genes

In [16]:
killerTcell = ['SYK','IFNGR1','SH2D1A','NFATC2','PIK3CD','ITGAL','PIK3CG','VAV1','ICAM1','ZAP70','FCGR3A','PPP3CC','LCK','PLCG2','RAC2','PTK2B','FAS','GRB2','LCP2','PTPN6']

killerTcells_column_names = []
for gene in killerTcell:
    gene += "_proteomics"
    killerTcells_column_names.append(gene)
len(killerTcells_column_names)


20

In [17]:
df_FDR_killerT = df_FDR_append[df_FDR_append.Comparison.isin(killerTcells_column_names)]
df_FDR_killerT

,Comparison,Correlation,P_value,Cancer Type
2,GRB2_proteomics,-0.610889,1.886384e-11,GBM
73,PLCG2_proteomics,-0.472453,7.918099e-07,GBM
75,SYK_proteomics,-0.470849,8.729180e-07,GBM
87,PIK3CD_proteomics,-0.466265,1.150323e-06,GBM
100,LCP2_proteomics,-0.460881,1.582634e-06,GBM
...,...,...,...,...
74534,RAC2_proteomics,-0.089875,3.504359e-01,Lscc
76994,PTK2B_proteomics,0.045180,6.393018e-01,Lscc
77727,FCGR3A_proteomics,0.032949,7.325622e-01,Lscc
78767,ICAM1_proteomics,0.016023,8.680469e-01,Lscc


In [18]:
p.plotCircleHeatMap(df_FDR_killerT,"P_value","Correlation","Comparison","Cancer Type")

/Users/Lindsey/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:30: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/Users/Lindsey/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:31: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/Users/Lindsey/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:32: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: 

Loading BokehJS ...

In [12]:
df_FDR_wide_killerTcells = df_FDR_wide[df_FDR_wide.Comparison.isin(killerTcells_column_names)]
df_FDR_wide_killerTcells

,Comparison,Correlation_Gbm,P_value_Gbm,Correlation_kidney,P_value_kidney,Correlation_Ovar,P_value_Ovar,Correlation_Brca,P_value_Brca,Correlation_Luad,P_value_Luad,Correlation_hnscc,P_value_hnscc,Correlation_colon,P_value_colon,Correlation_Lscc,P_value_Lscc
2,GRB2_proteomics,-0.610889,1.886384e-11,NaN,NaN,NaN,NaN,NaN,NaN,-0.302439,0.001321,-0.532341,2.559824e-09,NaN,NaN,NaN,NaN
73,PLCG2_proteomics,-0.472453,7.918099e-07,-0.317009,0.000740,NaN,NaN,0.268613,3.700474e-03,-0.408153,0.000010,-0.378950,4.837061e-05,NaN,NaN,NaN,NaN
75,SYK_proteomics,-0.470849,8.729180e-07,-0.552481,0.002806,NaN,NaN,NaN,NaN,NaN,NaN,-0.298428,1.622063e-03,NaN,NaN,NaN,NaN
87,PIK3CD_proteomics,-0.466265,1.150323e-06,NaN,NaN,NaN,NaN,NaN,NaN,-0.331760,0.000399,NaN,NaN,NaN,NaN,NaN,NaN
100,LCP2_proteomics,-0.460881,1.582634e-06,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-0.390098,2.750943e-05,NaN,NaN,NaN,NaN
126,FCGR3A_proteomics,-0.447148,3.485855e-06,NaN,NaN,NaN,NaN,0.255106,8.004745e-03,NaN,NaN,-0.341008,2.849943e-04,NaN,NaN,NaN,NaN
127,VAV1_proteomics,-0.447143,3.486858e-06,-0.285693,0.002483,NaN,NaN,NaN,NaN,NaN,NaN,-0.439464,1.745944e-06,NaN,NaN,NaN,NaN
270,ITGAL_proteomics,-0.412644,2.192297e-05,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-0.443845,1.337720e-06,NaN,NaN,NaN,NaN
319,PTPN6_proteomics,-0.402017,3.713191e-05,NaN,NaN,NaN,NaN,NaN,NaN,-0.371990,0.000063,-0.411321,8.866717e-06,NaN,NaN,NaN,NaN
363,PIK3CG_proteomics,-0.394048,5.449073e-05,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-0.402484,1.434139e-05,NaN,NaN,NaN,NaN


In [19]:
only_killerT = ['SYK','IFNGR1','SH2D1A','ITGAL','ICAM1','FCGR3A','PLCG2','RAC2','PTK2B','FAS']

In [20]:
killerTcells_column_names = []
for gene in only_killerT:
    gene += "_proteomics"
    killerTcells_column_names.append(gene)
len(killerTcells_column_names)

10

In [39]:
df_FDR_killerT = df_FDR_append[df_FDR_append.Comparison.isin(killerTcells_column_names)]
df_FDR_killerT

NameError: name 'killerTcells_column_names' is not defined

In [22]:
p.plotCircleHeatMap(df_FDR_killerT,"P_value","Correlation","Comparison","Cancer Type")

/Users/Lindsey/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:30: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/Users/Lindsey/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:31: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/Users/Lindsey/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:32: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: 

Loading BokehJS ...

In [15]:
cancer_genes = ['RB1','CTBP1','GNAI3','PDGFA','ADCY8','EDNRB','GNA12','PLCG1','CTNNA2','APC2','ARNT2','NOS2','ZBTB16','MMP2','BCR','CDK6','TRAF4','APC','TRAF6','GNAS','CTNNB1','FGFR3','FGFR2']
cancer_genes = ['GAMT','GCAT','GCSH','SRR','GATM','CBS','PSAT1','CTH','CHDH','PHGDH','ALDH7A1','DLD','PSPH']
cancer_genes = ['MAGI1','YES1','ACTN2','SHROOM2','GNAI3','CASK','MPP5','TJP1','CLDN10','PPP2R1A','EPB41L1','CTNNB1','CTNNA2','MPDZ','TJP2']

In [16]:
cancer_column_names = []
for gene in cancer_genes:
    gene += "_proteomics"
    cancer_column_names.append(gene)
len(cancer_column_names)

cancer = df_FDR_append[df_FDR_append.Comparison.isin(cancer_column_names)]
cancer

,Comparison,Correlation,P_value,Cancer Type
101,CTNNB1_proteomics,0.460836,0.000002,GBM
197,CLDN10_proteomics,0.425627,0.000011,GBM
335,MAGI1_proteomics,0.398807,0.000043,GBM
416,YES1_proteomics,0.383762,0.000088,GBM
420,PPP2R1A_proteomics,0.383320,0.000090,GBM
...,...,...,...,...
72068,ACTN2_proteomics,0.147024,0.125343,Lscc
74995,TJP1_proteomics,0.081162,0.399285,Lscc
76045,MPDZ_proteomics,-0.062346,0.517601,Lscc
76731,TJP2_proteomics,-0.050871,0.606296,Lscc


In [17]:
p.plotCircleHeatMap(cancer,"P_value","Correlation","Comparison","Cancer Type")

/Users/Lindsey/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:31: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/Users/Lindsey/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:32: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/Users/Lindsey/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:33: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: 

Loading BokehJS ...